In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import string
import nltk
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
# from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Downloading package stopwords to /home/priya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#read the data set
data = pd.read_csv("./Data/phrases.csv")
# data = pd.read_csv("./Data/phrases.csv")
# data=pd.DataFrame(data)
# print type(data)
# print data.shape
data=data.dropna()
print data['Emotion'].unique()
# print data.shape

['joy' 'fear' 'anger' 'sadness' 'disgust' 'shame' 'guilt']


## Preprocess the data set via cleaning, tokenisation and lemmatization

In [3]:
# print data
import re

def clean_dataset(data):
    translator = string.maketrans('', '')
    for index,row in data.iterrows():
#         print index,row['Phrase']
        row['Phrase'] = row['Phrase'].replace('[','')
        row['Phrase'] = row['Phrase'].replace(']','')
        row['Phrase'] = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', row['Phrase'])
        row['Phrase'] = re.sub('@[^\s]+','USER', row['Phrase'])
        row['Phrase'] = row['Phrase'].lower().replace("ё", "е")
        row['Phrase'] = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', row['Phrase'])
        row['Phrase'] = re.sub(' +',' ', row['Phrase'])
        row['Phrase'] = row['Phrase'].strip()
        row['Phrase'] = row['Phrase'].translate(translator,string.punctuation)
#         print row['Phrase']
    return data
data = clean_dataset(data)
print data.head()

   Emotion                                             Phrase
0      joy  on days when i feel closing to my partner and ...
1     fear  every time i imagine that someone i love or i ...
2    anger  when i had been obviously unjustly treated and...
3  sadness  when i think about the short time that we live...
4  disgust  at a gathering i found myself involuntarily si...


In [4]:
def bag_of_words(data):
    count_vect=CountVectorizer()
    count_vect.fit(data) #creates vocab of words
#     print (count_vect.vocabulary_)
    data_phrase=count_vect.transform(data)
#     print(data_phrase.shape)
#     print(type(data_phrase))
#     print(data_phrase.toarray())
    return data_phrase,count_vect
data_phrase,count_vect=bag_of_words(data['Phrase'])
data_phrase.shape

(7652, 8930)

In [5]:
def tf_idf(data_phrase):
    tfidf_transformer = TfidfTransformer()
    phrase_tfidf = tfidf_transformer.fit_transform(data_phrase)
    return phrase_tfidf.toarray(),tfidf_transformer
phrase_tfidf,tfidf_transformer=tf_idf(data_phrase)
phrase_tfidf.shape

(7652, 8930)

In [6]:
# Import LabelEncoder
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
emotion_data=data['Emotion'].values
emo_enc=le.fit_transform(emotion_data)
print emo_enc

[4 2 0 ... 1 6 3]


In [7]:
train_phrase, validate_phrase = np.split(phrase_tfidf,[int(.8*len(phrase_tfidf))])
# train_label,validate_label=np.split(emotion_data,[int(.8*len(emotion_data))])
train_label,validate_label=np.split(emo_enc,[int(.8*len(emo_enc))])
print "data",train_phrase.shape,validate_phrase.shape
print "label",train_label.shape,validate_label.shape

data (6121, 8930) (1531, 8930)
label (6121,) (1531,)


In [ ]:
modelknn = KNeighborsClassifier(n_neighbors=3)
modelknn.fit(train_phrase, train_label)
label_predicted = modelknn.predict(validate_phrase)
 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

sklearn_score_train = modelknn.score(train_phrase,train_label)
print("Sklearn's score on training data :",sklearn_score_train)
sklearn_score_test = modelknn.score(validate_phrase,validate_label)
print("Sklearn's score on testing data :",sklearn_score_test)
print("Classification report for testing data :-")
print(classification_report(validate_label, label_predicted))

In [ ]:
def data_preprocess(text):
#     cleaning
    translator = string.maketrans('', '')
    text = text.replace('[','')
    text = text.replace(']','')
    text = text.strip()
    text = text.translate(translator,string.punctuation)
#     stop word removal
    stops = set(stopwords.words("english"))
    text = text.lower().split(' ')
    text = [w.strip() for w in text if not w in stops and len(w) >= 2]
    text = " ".join(text)
#     lemmatization
    stemmer = SnowballStemmer('english')
    text = text.split()
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    
    return text

In [ ]:
test_sentence=raw_input("Enter text:")
test_sen=[]
test_sen.append(data_preprocess(test_sentence))
bow=count_vect.transform(test_sen)
tf_test=tfidf_transformer.transform(bow).toarray()
predicted_test= modelknn.predict(tf_test)
le.inverse_transform(predicted_test)